In [1]:


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, PolynomialFeatures
from xgboost import XGBRegressor 
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import logging
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List
from utils import FeatureEngineer
from scipy import stats 

In [2]:
# Setup logging
logging.basicConfig(
    filename='model_training.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filemode='w'
)
logging.info('Training started on %s', pd.Timestamp.now())

# Load data
try:
    df: pd.DataFrame = pd.read_csv('Health_index1.csv')
    df.columns = df.columns.str.strip()
    logging.info(f'Loaded Health_index1.csv. Shape: {df.shape}')
except FileNotFoundError:
    logging.error('Health_index1.csv not found.')
    raise

In [3]:

# Feature engineering
df = FeatureEngineer.add_gas_ratios(df)
df = FeatureEngineer.add_log_features(df)

# Add Doernenburg and Rogers ratios
df['CH4_H2_ratio'] = df['Methane'] / (df['Hydrogen'] + 1)
df['C2H2_C2H4_ratio'] = df['Acetylene'] / (df['Ethylene'] + 1)
df['C2H2_CH4_ratio'] = df['Acetylene'] / (df['Methane'] + 1)
df['C2H6_C2H2_ratio'] = df['Ethane'] / (df['Acetylene'] + 1)

# Add gas percentages and concentrations
df['Total_Gases'] = df[['Hydrogen', 'Methane', 'CO', 'CO2', 'Ethylene', 'Ethane', 'Acetylene']].sum(axis=1)
df['H2_percent'] = df['Hydrogen'] / df['Total_Gases'] * 100
df['CH4_percent'] = df['Methane'] / df['Total_Gases'] * 100
df['CO_percent'] = df['CO'] / df['Total_Gases'] * 100
df['CO2_percent'] = df['CO2'] / df['Total_Gases'] * 100
df['Gas_Concentration'] = df['Total_Gases'] / 1000000

# Add quality indicator interactions
df['Gas_DBDS_interaction'] = df['Total_Gases'] * df['DBDS']
df['Gas_Water_interaction'] = df['Total_Gases'] * df['Water content']
df['Power_Gas_interaction'] = df['Total_Gases'] * df['Power factor']

# Add binned features
df['H2_level'] = pd.qcut(df['Hydrogen'], q=5, labels=['VL', 'L', 'M', 'H', 'VH'])
df['Gas_level'] = pd.qcut(df['Total_Gases'], q=5, labels=['VL', 'L', 'M', 'H', 'VH'])
df['Water_level'] = pd.qcut(df['Water content'], q=5, labels=['VL', 'L', 'M', 'H', 'VH'])

# Convert categorical to dummy variables
df = pd.get_dummies(df, columns=['H2_level', 'Gas_level', 'Water_level'])

# Add polynomial features
gas_cols = ['Hydrogen', 'Methane', 'CO', 'CO2', 'Ethylene']
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_features = poly.fit_transform(df[gas_cols])
poly_feature_names = [f'Poly_{i}' for i in range(poly_features.shape[1])]
df_poly = pd.DataFrame(poly_features, columns=poly_feature_names)

# Combine all features
df = pd.concat([df, df_poly], axis=1)

# Define features for regression
regression_features: List[str] = [
    'Hydrogen', 'Oxigen', 'Nitrogen', 'Methane', 'CO', 'CO2', 'Ethylene',
    'Ethane', 'Acetylene', 'DBDS', 'Power factor', 'Interfacial V',
    'Dielectric rigidity', 'Water content',
    'CH4_H2_ratio', 'C2H2_C2H4_ratio', 'C2H2_CH4_ratio', 'C2H6_C2H2_ratio',
    'H2_percent', 'CH4_percent', 'CO_percent', 'CO2_percent',
    'Gas_Concentration', 'Gas_DBDS_interaction', 'Gas_Water_interaction',
    'Power_Gas_interaction'
]

# Verify no target variables are in features
assert 'Health_index' not in regression_features, "Health_index should not be a feature"
assert 'Life_expectation' not in regression_features, "Life_expectation should not be a feature"

# Add dummy variable columns
level_columns = [col for col in df.columns if col.endswith('_VL') or 
                                            col.endswith('_L') or 
                                            col.endswith('_M') or 
                                            col.endswith('_H') or 
                                            col.endswith('_VH')]
regression_features.extend(level_columns)
regression_features.extend(poly_feature_names)

# Remove outliers function
def remove_outliers(X, y, z_threshold=3):
    z_scores = np.abs(stats.zscore(X))
    mask = (z_scores < z_threshold).all(axis=1)
    return X[mask], y[mask]

# Prepare and clean regression data
X_health: pd.DataFrame = df[regression_features].fillna(0.0)
y_health: pd.Series = df['Health_index'].astype(float)
X_life: pd.DataFrame = df[regression_features].fillna(0.0)
y_life: pd.Series = df['Life_expectation'].astype(float)

# Convert boolean columns to float to avoid dtype issues in zscore
for col in X_health.select_dtypes(include=['bool']).columns:
    X_health[col] = X_health[col].astype(float)
for col in X_life.select_dtypes(include=['bool']).columns:
    X_life[col] = X_life[col].astype(float)

# Remove outliers
X_health_clean, y_health_clean = remove_outliers(X_health, y_health)
X_life_clean, y_life_clean = remove_outliers(X_life, y_life)

# Scale features using RobustScaler
scaler = RobustScaler()
X_health_scaled = scaler.fit_transform(X_health_clean)
X_life_scaled = scaler.transform(X_life_clean)

In [4]:

# Scale features
scaler: StandardScaler = StandardScaler()
X_health_scaled: np.ndarray = scaler.fit_transform(X_health)
X_life_scaled: np.ndarray = scaler.transform(X_life)
joblib.dump(scaler, 'scaler_fitted_on_full_data.gz')
logging.info('Saved scaler.')

# Split data
X_health_train: np.ndarray
X_health_test: np.ndarray
y_health_train: pd.Series
y_health_test: pd.Series
X_health_train, X_health_test, y_health_train, y_health_test = train_test_split(
    X_health_scaled, y_health, test_size=0.2, random_state=42
)
X_life_train: np.ndarray
X_life_test: np.ndarray
y_life_train: pd.Series
y_life_test: pd.Series
X_life_train, X_life_test, y_life_train, y_life_test = train_test_split(
    X_life_scaled, y_life, test_size=0.2, random_state=42
)
logging.info(f'Training set sizes: Health: {X_health_train.shape}, Life: {X_life_train.shape}')


In [5]:
"""# Feature selection using feature importance
def select_top_features(model, feature_names, threshold=0.01):  # Increased threshold
    importance = model.feature_importances_
    selected = [i for i, imp in enumerate(importance) if imp > threshold]
    return [feature_names[i] for i in selected], selected

# Define optimized XGBoost parameters with more conservative settings
xgb_params = {
    'n_estimators': 1500,       # Balanced for current features
    'max_depth': 4,             # Slightly increased
    'learning_rate': 0.01,      # Kept moderate
    'min_child_weight': 5,      # Balanced
    'subsample': 0.8,           # Increased for stability
    'colsample_bytree': 0.8,    # Increased for better feature use
    'gamma': 0.1,               # Reduced for better splits
    'reg_alpha': 0.2,           # Reduced L1 regularization
    'reg_lambda': 1.0,          # Moderate L2 regularization
    'random_state': 42,
    'tree_method': 'hist',      
    'objective': 'reg:squarederror',
    'max_leaves': 24            # Increased for better learning
}

# Train initial model for feature selection
initial_health_model = XGBRegressor(**xgb_params)
initial_health_model.fit(X_health_train, y_health_train)

# Select important features
top_features, feature_idx = select_top_features(initial_health_model, regression_features)
logging.info(f'Selected {len(top_features)} important features')

# Train final models with selected features
X_health_train_selected = X_health_train[:, feature_idx]
X_health_test_selected = X_health_test[:, feature_idx]
X_life_train_selected = X_life_train[:, feature_idx]
X_life_test_selected = X_life_test[:, feature_idx]

# Train Health_index regressor
health_model = XGBRegressor(**xgb_params)
health_model.fit(
    X_health_train_selected, 
    y_health_train,
    eval_set=[(X_health_train_selected, y_health_train), 
              (X_health_test_selected, y_health_test)],
    verbose=True
)

# Train Life_expectation regressor
life_model = XGBRegressor(**xgb_params)
life_model.fit(
    X_life_train_selected, 
    y_life_train,
    eval_set=[(X_life_train_selected, y_life_train), 
              (X_life_test_selected, y_life_test)],
    verbose=True
)

# Evaluate models
health_pred = health_model.predict(X_health_test_selected)
health_mse = mean_squared_error(y_health_test, health_pred)
health_cv_mse = -cross_val_score(
    health_model, X_health_train_selected, y_health_train, 
    cv=5, 
    scoring='neg_mean_squared_error'
)

life_pred = life_model.predict(X_life_test_selected)
life_mse = mean_squared_error(y_life_test, life_pred)
life_cv_mse = -cross_val_score(
    life_model, X_life_train_selected, y_life_train, 
    cv=5, 
    scoring='neg_mean_squared_error'
)""""fault analysis model.ipynb"

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV

# Updated feature selection with stricter threshold
def select_top_features(model, feature_names, threshold=0.02):  # Increased threshold
    importance = model.feature_importances_
    selected = [i for i, imp in enumerate(importance) if imp > threshold]
    selected_features = [feature_names[i] for i in selected]
    
    logging.info(f"Selected features with importance > {threshold}")
    for feat, imp in zip(selected_features, importance[selected]):
        logging.info(f"Feature: {feat}, Importance: {imp:.4f}")
    
    return selected_features, selected

# Optimized XGBoost parameters based on feature importance
xgb_params = {
    'n_estimators': 200,       # Increased for better convergence
    'max_depth': 4,             # Kept moderate to prevent overfitting
    'learning_rate': 0.01,     # Reduced for better generalization
    'min_child_weight': 7,      # Increased to prevent overfitting
    'subsample': 0.7,           # Reduced to prevent overfitting
    'colsample_bytree': 0.7,    # Reduced to focus on important features
    'gamma': 0.1,               # Increased for more conservative splits
    'reg_alpha': 0.4,           # Increased L1 regularization
    'reg_lambda': 1.5,          # Increased L2 regularization
    'random_state': 42,
    'tree_method': 'hist',
    'objective': 'reg:squarederror',
    'max_leaves': 24            # Reduced to prevent overfitting
}

# Create focused feature interactions
def create_focused_interactions(df):
    return pd.DataFrame({
        'DBDS_Acetylene': df['DBDS'] * df['Acetylene'],
        'Methane_Acetylene': df['Methane'] * df['Acetylene'],
        'DBDS_Gas_Ratio': df['DBDS'] / (df['Total_Gases'] + 1),
        'Acetylene_Ratio': df['Acetylene'] / (df['Total_Gases'] + 1)
    })

# Model evaluation function with detailed metrics
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    cv_scores = -cross_val_score(
        model, X_train, y_train,
        cv=5,
        scoring='neg_mean_squared_error'
    )
    
    logging.info(f"\n{model_name} Performance Metrics:")
    logging.info(f"MSE (Test): {mse:.4f}")
    logging.info(f"RMSE (Test): {rmse:.4f}")
    logging.info(f"MAE (Test): {mae:.4f}")
    logging.info(f"R² Score: {r2:.4f}")
    logging.info(f"CV MSE: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
    
    return mse, cv_scores

# Train and evaluate models
initial_health_model = XGBRegressor(**xgb_params)
initial_health_model.fit(X_health_train, y_health_train)

# Select features and train final models
top_features, feature_idx = select_top_features(initial_health_model, regression_features)
X_health_train_selected = X_health_train[:, feature_idx]
X_health_test_selected = X_health_test[:, feature_idx]
X_life_train_selected = X_life_train[:, feature_idx]
X_life_test_selected = X_life_test[:, feature_idx]

# Train final models
health_model = XGBRegressor(**xgb_params)
health_model.fit(
    X_health_train_selected, 
    y_health_train,
    eval_set=[(X_health_train_selected, y_health_train), 
              (X_health_test_selected, y_health_test)],
    verbose=True
)

life_model = XGBRegressor(**xgb_params)
life_model.fit(
    X_life_train_selected, 
    y_life_train,
    eval_set=[(X_life_train_selected, y_life_train), 
              (X_life_test_selected, y_life_test)],
    verbose=True
)

# Evaluate final models
health_metrics = evaluate_model(
    health_model, X_health_train_selected, X_health_test_selected,
    y_health_train, y_health_test, "Health Index"
)

life_metrics = evaluate_model(
    life_model, X_life_train_selected, X_life_test_selected,
    y_life_train, y_life_test, "Life Expectation"
)

[0]	validation_0-rmse:17.39067	validation_1-rmse:18.48817
[1]	validation_0-rmse:17.27327	validation_1-rmse:18.36746
[2]	validation_0-rmse:17.15863	validation_1-rmse:18.24114
[3]	validation_0-rmse:17.06207	validation_1-rmse:18.15179
[4]	validation_0-rmse:16.94515	validation_1-rmse:18.03704
[5]	validation_0-rmse:16.83014	validation_1-rmse:17.92664
[6]	validation_0-rmse:16.73397	validation_1-rmse:17.82242
[7]	validation_0-rmse:16.63262	validation_1-rmse:17.72035
[8]	validation_0-rmse:16.53427	validation_1-rmse:17.61362
[9]	validation_0-rmse:16.42947	validation_1-rmse:17.50932
[10]	validation_0-rmse:16.32238	validation_1-rmse:17.39498
[11]	validation_0-rmse:16.21656	validation_1-rmse:17.28469
[12]	validation_0-rmse:16.11675	validation_1-rmse:17.19627
[13]	validation_0-rmse:16.01859	validation_1-rmse:17.10132
[14]	validation_0-rmse:15.91723	validation_1-rmse:16.99488
[15]	validation_0-rmse:15.82442	validation_1-rmse:16.91320
[16]	validation_0-rmse:15.73084	validation_1-rmse:16.83293
[17]	va

In [6]:
# 
# Plot feature importance for regression
plt.figure(figsize=(12, 8))
sns.barplot(x=health_model.feature_importances_, y=top_features)
plt.title('Feature Importance for Health Index')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('health_feature_importance.png')
plt.close()
logging.info('Saved Health_index feature importance plot.')

plt.figure(figsize=(12, 8))
sns.barplot(x=life_model.feature_importances_, y=top_features)
plt.title('Feature Importance for Life Expectation')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('life_feature_importance.png')
plt.close()
logging.info('Saved Life_expectation feature importance plot.')



In [7]:
# Summary
logging.info('Training completed. Summary:')
logging.info(f'- Health Index MSE (Test): {health_metrics[0]:.4f}, CV MSE: {health_metrics[1].mean():.4f} ± {health_metrics[1].std():.4f} (Target: ≤50)')
logging.info(f'- Life Expectation MSE (Test): {life_metrics[0]:.4f}, CV MSE: {life_metrics[1].mean():.4f} ± {life_metrics[1].std():.4f} (Target: ≤30)')
print('Training completed. Check model_training.log for details.')

Training completed. Check model_training.log for details.
